## 1.Stratergy

In [73]:
from fyers_api import fyersModel
import os
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import warnings
warnings. simplefilter(action='ignore', category=Warning)

with open("access.txt", 'r') as r:
    access_token = r.read()

client_id = 'L0QBIDUTN6-100'
fyers = fyersModel.FyersModel(token=access_token, log_path=os.getcwd(), client_id=client_id)

def crossover(stock, type, timeframe, duration):
    symbol = "NSE:" + stock + "-" + type
    start = dt.date.today() - dt.timedelta(duration)
    end = dt.date.today()
    
    sdata = pd.DataFrame()
    
    while start <= end:
        end2 = start + dt.timedelta(days=60)  
        
        data = {"symbol": symbol,
                "resolution": timeframe,
                "date_format": "1", 
                "range_from": start, 
                "range_to": end2,
                "cont_flag": "1"
                }
        candles = fyers.history(data)
        
        if candles.get('candles'):
            s = pd.DataFrame(candles['candles'])
            sdata = pd.concat([sdata, s], ignore_index=True)
        
        start = end2 + dt.timedelta(1)
    
    sdata.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    sdata['date'] = pd.to_datetime(sdata['date'], unit='s')
    sdata['date'] = sdata['date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
    sdata['date'] = sdata['date'].dt.tz_localize(None)
    sdata = sdata.set_index('date')
    
    sdata['20_SMA'] = sdata['close'].rolling(window=20, min_periods=1).mean()
    sdata['50_SMA'] = sdata['close'].rolling(window=50, min_periods=1).mean()
    sdata['signal'] = 0
    sdata['signal'] = np.where(sdata['20_SMA'] > sdata['50_SMA'], 1, 0)
    sdata['Position'] = sdata['signal'].diff()
    
    
    df_pos = sdata[(sdata['Position'] == 1) | (sdata['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    
    return sdata, df_pos

# Call the function and store the result in DataFrames
data, df_pos = crossover('HDFCBANK', 'EQ', '30', 100)
# Now you can use 'data' and 'df_pos' DataFrames

df_pos


,open,high,low,close,volume,20_SMA,50_SMA,signal,Position
date,,,,,,,,,
2023-08-28 14:15:00,1579.10,1582.00,1577.70,1579.50,1740461,1569.5175,1569.150,1,Buy
2023-08-31 13:45:00,1573.95,1574.00,1570.45,1570.85,1624138,1580.1300,1581.403,0,Sell
2023-09-04 15:15:00,1584.95,1585.25,1583.05,1584.20,681194,1577.7600,1577.360,1,Buy
2023-09-18 14:45:00,1632.00,1632.40,1626.50,1627.60,2957433,1645.1025,1645.738,0,Sell
2023-10-05 10:45:00,1535.65,1542.95,1534.60,1542.90,1159235,1523.7400,1523.393,1,Buy
2023-10-09 13:15:00,1514.85,1515.05,1513.65,1514.65,487298,1529.3375,1530.096,0,Sell
2023-10-11 13:45:00,1532.35,1532.80,1529.25,1532.20,1227767,1528.2125,1527.763,1,Buy
2023-10-16 12:15:00,1531.70,1533.00,1528.50,1531.85,564504,1535.8800,1535.945,0,Sell
2023-10-18 09:45:00,1532.45,1535.00,1528.60,1534.15,1235299,1538.1900,1538.186,1,Buy


In [44]:
# required_df = df_pos[(df_pos['Position'] != 'Sell').cumsum().gt(0) & (df_pos['Position'] != 'Buy').cumsum().lt(df_pos['Position'].eq('Buy').cumsum().max())]
# required_df

## 2. Making Stats

In [74]:
# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price
class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        self.backtesting = self.backtesting.append(self.trade_log, ignore_index=True)
    
    def stats(self):
        df = self.backtesting
        parameters = ['Total Trade Scripts', 'Total Trade', 'PNL',  'Winners', 'Losers', 'Win Ratio','Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Average PNL Per Trade', 'Risk Reward']
        total_traded_scripts = len(df['Equity Name'].unique())
        total_trade = len(df.index)
        pnl = df.PNL.sum()
        winners = len(df[df.PNL > 0])
        loosers = len(df[df.PNL <= 0])
        win_ratio = str(round((winners/total_trade) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
        average_loss_per_trade = round(total_loss/loosers, 2)
        average_profit_per_trade = round(total_profit/winners, 2)
        average_pnl_per_trade = round(pnl/total_trade, 2)
        risk_reward = f'1:{-1 * round(average_profit_per_trade/average_loss_per_trade, 2)}'
        data_points = [total_traded_scripts,total_trade,pnl,winners, loosers, win_ratio, total_profit, total_loss, average_loss_per_trade, average_profit_per_trade, average_pnl_per_trade, risk_reward]
        data = list(zip(parameters,data_points ))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='psql'))

## 3. Calling Stats & Stratergy

In [75]:
bt = Backtest()
capital = 50000
scripts = ['HDFCBANK','TCS']
for stock in scripts:
    data = crossover('HDFCBANK', 'EQ', '5', 100)
    required_df = df_pos[(df_pos['Position'] != 'Sell').cumsum().gt(0) & (df_pos['Position'] != 'Buy').cumsum().lt(df_pos['Position'].eq('Buy').cumsum().max())]
    for index, data in required_df.iterrows():
        if(data.Position == 'Buy'):
            qty = capital // data.open
            bt.buy(stock, index, data.open, qty)
        else:
            bt.sell(index, data.open, 'Exit Trigger', 0)

df = bt.backtesting

# bt.backtesting.to_csv('Backtest.csv')

In [72]:
bt.stats()

## INCLUDES BUY & SELL

+--------------------------+------------------+
| Parameters               | Values           |
|--------------------------+------------------|
| Total Trade Scripts      | 2                |
| Total Trade              | 22               |
| PNL                      | 90.7999999999999 |
| Winners                  | 6                |
| Losers                   | 16               |
| Win Ratio                | 27.27%           |
| Total Profit             | 4082.0           |
| Total Loss               | -3991.2          |
| Average Loss per Trade   | -249.45          |
| Average Profit per Trade | 680.33           |
| Average PNL Per Trade    | 4.13             |
| Risk Reward              | 1:2.73           |
+--------------------------+------------------+
